# Merge CTD and satellite data extracted at observation locations
Created by Ivan Lima on Tue Jan 31 2023 19:39:58 -0500

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import datetime, warnings, os
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

Last updated on Sun Feb 12 09:56:06 2023


In [2]:
pd.options.display.max_columns = 50
warnings.filterwarnings('ignore')

## Read satellite & CTD data

In [3]:
# Read & concatenate annual satellite data files into one
datadir = 'data/by_year'
filelist = sorted([os.path.join(datadir, file) for file in os.listdir(datadir)])
df_list = []
for file in filelist:
    df_list.append(pd.read_csv(file, parse_dates=['Date'], index_col=0))
    
df_sat = pd.concat(df_list, axis=0)
df_sat.head()

,Cast,Latitude,Longitude,Year,Month,Day,Platform_Type,Date,ADT,SLA,SST,SST_hires,Chl,KD490
104206,10498917,44.266701,-63.316700,2000,1,10,1,2000-01-10,-0.118689,0.017700,4.510375,NaN,0.503149,0.061657
104207,10498929,45.046700,-66.029198,2000,1,19,1,2000-01-19,-0.167368,-0.022110,4.719816,NaN,2.728127,0.158278
104208,10498931,44.980999,-67.016197,2000,1,19,1,2000-01-19,-0.171099,-0.029850,5.489990,NaN,3.105249,0.199335
104209,10498941,44.933300,-66.849998,2000,1,28,1,2000-01-28,-0.114187,0.027205,3.840302,NaN,2.565306,0.152913
104210,10498944,45.078300,-67.097801,2000,1,28,1,2000-01-28,-0.121198,0.021531,4.040009,NaN,5.620232,0.249472


In [4]:
ds_ctd = xr.open_dataset('/home/ivan/Projects/Bottom_water_BGC/data/CombinedCTD12_8_2021.nc')
df_ctd = ds_ctd.to_dataframe()
df_ctd = df_ctd.droplevel(0) # drop "column" index
df_ctd.index.name = None
df_ctd = df_ctd[df_ctd.Date_YYYYMMDD!=19220100] # remove invalid dates
df_ctd['Date'] = pd.to_datetime(df_ctd['Date_YYYYMMDD'], format='%Y%m%d')
df_ctd = df_ctd.drop(['Date_YYYYMMDD','Time_DaysSince1_1_1770'], axis=1)
df_ctd = df_ctd[df_ctd.Date.dt.year > 1980] # only data from satellite era
col_names = {
    'CastID':'Cast',
    'Lat':'Latitude',
    'Lon':'Longitude',
    'Depth_m':'Depth',
    'Temp_C':'Temperature',
    'Sal_PSU':'Salinity',
    'DO_umolkg':'Oxygen',
    'Temp_Flag':'Temp_flag',
    'Sal_Flag':'Salt_flag'
}
df_ctd = df_ctd.rename(columns=col_names) # rename columns
df_ctd.head()

,Cast,Latitude,Longitude,Depth,Temperature,Temp_flag,Salinity,Salt_flag,Oxygen,DO_flag,Platform_Type,pCO2_year,pCO2_month,Date
864823,3344598.0,41.450001,-64.150002,19.260000,NaN,NaN,32.700001,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864824,3344598.0,41.450001,-64.150002,20.959999,NaN,NaN,32.700001,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864825,3344598.0,41.450001,-64.150002,23.230000,NaN,NaN,32.701000,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864826,3344598.0,41.450001,-64.150002,24.850000,NaN,NaN,32.701000,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864827,3344598.0,41.450001,-64.150002,26.830000,NaN,NaN,32.701000,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05


## Do inner join 

In [6]:
sat_cols = ['Cast', 'Latitude', 'Longitude', 'Date', 'ADT', 'SLA', 'SST', 'SST_hires', 'Chl', 'KD490']
ctd_cols = ['Cast', 'Latitude', 'Longitude', 'Date', 'Depth', 'Temperature', 'Salinity', 'Oxygen',
            'pCO2_year', 'pCO2_month', 'Platform_Type', 'Temp_flag', 'Salt_flag', 'DO_flag']
df_merged = pd.merge(df_ctd[ctd_cols], df_sat[sat_cols], on=['Cast'], how='inner', suffixes=['_ctd', '_sat'])

print('Original data: {:,} rows'.format(len(df_ctd)))
print('New merged data: {:,} rows'.format(len(df_merged)))
print('Difference: {:,} rows'.format(len(df_merged)-len(df_ctd)))

Original data: 26,330,017 rows
New merged data: 11,681,025 rows
Difference: -14,648,992 rows


## Check if records match for merged variables 

In [7]:
def check(vname):
    vname_ctd, vname_sat = vname+'_ctd', vname+'_sat'
    df_work = df_merged[[vname_ctd, vname_sat]].dropna()
    if len(df_work) == (df_work[vname_ctd]==df_work[vname_sat]).sum():
        print('{}: OK'.format(vname))
    else:
        n = len(df_work) - (df_work[vname_ctd]==df_work[vname_sat]).sum()
        print('{}: {:,} records do not match'.format(vname, n))

varcols = [c[:-4] for c in df_merged.columns if c.endswith('_ctd')]
for vname in varcols:
    check(vname)

Latitude: 10,503,514 records do not match
Longitude: 9,528,058 records do not match
Date: OK


**Note:** The *Date* matches for all records but *Latitude* and *Longitude* don't due to differences in precision.

In [8]:
df_merged[['Date_ctd','Date_sat','Latitude_ctd','Latitude_sat','Longitude_ctd','Longitude_sat']].head()

,Date_ctd,Date_sat,Latitude_ctd,Latitude_sat,Longitude_ctd,Longitude_sat
0,2000-01-10,2000-01-10,44.266701,44.266701,-63.3167,-63.3167
1,2000-01-10,2000-01-10,44.266701,44.266701,-63.3167,-63.3167
2,2000-01-10,2000-01-10,44.266701,44.266701,-63.3167,-63.3167
3,2000-01-10,2000-01-10,44.266701,44.266701,-63.3167,-63.3167
4,2000-01-10,2000-01-10,44.266701,44.266701,-63.3167,-63.3167


In [9]:
lat_diff  = df_merged.Latitude_ctd - df_merged.Latitude_sat
lat_diff.min(), lat_diff.max()

(-4.973799150320701e-14, 4.973799150320701e-14)

In [10]:
lon_diff  = df_merged.Longitude_ctd - df_merged.Longitude_sat
lon_diff.min(), lon_diff.max()

(-4.973799150320701e-14, 5.684341886080802e-14)

## Save merged data set to HDF file 

In [11]:
df_merged_final = df_merged.drop(['Latitude_sat','Longitude_sat','Date_sat'], axis=1).rename(columns={
    'Latitude_ctd':'Latitude',
    'Longitude_ctd':'Longitude',
    'Date_ctd':'Date'})
yr_min, yr_max = df_merged_final.Date.dt.year.min(), df_merged_final.Date.dt.year.max()
outfile = 'data/CombinedCTD_satellite_{}-{}.h5'.format(yr_min, yr_max)
print('writing {}'.format(outfile))
df_merged_final.to_hdf(outfile, key='df_ctd_sat', mode='w', complevel=9)

writing data/CombinedCTD_satellite_2000-2021.h5
